<a href="https://colab.research.google.com/github/mit1280/fined-tuning/blob/main/phi_2_classification_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U accelerate peft bitsandbytes transformers einops datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-fram

In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          TrainingArguments,
                          Trainer)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)

from datasets import load_dataset

## Load train data

In [3]:
dataset_name = 'financial_phrasebank'
# load dataset
dataset = load_dataset(dataset_name, "sentences_50agree", split = ["train"])[0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]

In [4]:
dataset

Dataset({
    features: ['sentence', 'label'],
    num_rows: 4846
})

## Set label ids

In [ ]:
labels = dataset.features['label'].names
print(labels)

['negative', 'neutral', 'positive']


In [ ]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
print(id2label)

{0: 'negative', 1: 'neutral', 2: 'positive'}


## Load model and tokenizer

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["dense",
    "q_proj",
    "k_proj",
    "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 10,485,760 || all params: 2,790,169,600 || trainable%: 0.37581084676716425


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", padding=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Tokenize data

In [ ]:
dataset

Dataset({
    features: ['sentence', 'label'],
    num_rows: 4846
})

In [ ]:
data_df = dataset.to_pandas()

conversation_list = []

# Iterate over the DataFrame rows
for index, row in data_df.iterrows():
    # Construct the conversation list dynamically
    conversation = [
        {"from": "human", "value": f"Text: {row['sentence']}"},
        {"from": "phi", "value": "I've read this text."},
        {"from": "human", "value": "Please determine the sentiment of the given text and choose from the options: Positive, Negative, Neutral, or Cannot be determined."},
        {"from": "phi", "value": f"Sentiment of given text is {row['label']}"},
    ]
    # Append the conversation to the list
    conversation_list.append(conversation)

# Add the conversation list as a new column in the DataFrame
data_df['conversation'] = conversation_list

In [ ]:
template = tokenizer.default_chat_template
template = template.replace("role", "from").replace("content", "value")  # Change the system token
tokenizer.chat_template = template  # Set the new template
tokenizer.chat_template


No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format (without BOS/EOS tokens!). If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



"{% for message in messages %}{{'<|im_start|>' + message['from'] + '\n' + message['value'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [ ]:
data_df['chat_conversations'] = data_df['conversation'].apply(lambda x: tokenizer.apply_chat_template(x, tokenize=False))

In [ ]:
new_dataset = Dataset.from_pandas(data_df)

In [ ]:
def tokenize(sample):
    tokenized_text =  tokenizer(sample["chat_conversations"], padding=True, truncation=True, max_length=512)
    return tokenized_text

In [ ]:
tokenized_data = new_dataset.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=new_dataset.column_names)

tokenized_data

Tokenizing data:   0%|          | 0/4846 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 4846
})

In [ ]:
tokenizer.decode(tokenized_data[0]['input_ids'])

"<|im_start|>human\nText: According to Gran, the company has no plans to move all production to Russia, although that is where the company is growing.<|im_end|>\n<|im_start|>phi\nI've read this text.<|im_end|>\n<|im_start|>human\nPlease determine the sentiment of the given text and choose from the options: Positive, Negative, Neutral, or Cannot be determined.<|im_end|>\n<|im_start|>phi\nSentiment of given text is 1<|im_end|>\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext

## Training

In [ ]:
training_arguments = TrainingArguments(
        output_dir="phi-2-classification-sentiment",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        max_steps=10000,
        num_train_epochs=5
    )

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig

trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

Step,Training Loss
100,0.921700
200,0.674000
300,0.684500
400,0.687200
500,0.672500
600,0.675400
700,0.658900
800,0.658800
900,0.663800
1000,0.640500


Step,Training Loss
100,0.921700
200,0.674000
300,0.684500
400,0.687200
500,0.672500
600,0.675400
700,0.658900
800,0.658800
900,0.663800
1000,0.640500


TrainOutput(global_step=10000, training_loss=0.529747783279419, metrics={'train_runtime': 9559.7962, 'train_samples_per_second': 4.184, 'train_steps_per_second': 1.046, 'total_flos': 1.4279513657856e+17, 'train_loss': 0.529747783279419, 'epoch': 8.25})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.save_model("./classification")

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
)
from peft import LoraConfig, PeftModel

temp_model = PeftModel.from_pretrained(base_model, "./classification")
temp_model = temp_model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
temp_model.save_pretrained("phi-2-classification-sentiment-merged")
tokenizer.save_pretrained("phi-2-classification-sentiment-merged")

('phi-2-classification-sentiment-merged/tokenizer_config.json',
 'phi-2-classification-sentiment-merged/special_tokens_map.json',
 'phi-2-classification-sentiment-merged/vocab.json',
 'phi-2-classification-sentiment-merged/merges.txt',
 'phi-2-classification-sentiment-merged/added_tokens.json',
 'phi-2-classification-sentiment-merged/tokenizer.json')

In [ ]:
temp_model.push_to_hub("phi-2-classification-sentiment-merged", use_temp_dir=False)
tokenizer.push_to_hub("phi-2-classification-sentiment-merged", use_temp_dir=False)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mit1208/phi-2-classification-sentiment-merged/commit/49d441b73cff032c609240acb18f0778de64c319', commit_message='Upload tokenizer', commit_description='', oid='49d441b73cff032c609240acb18f0778de64c319', pr_url=None, pr_revision=None, pr_num=None)

# Inference

In [4]:
dataset_name = 'financial_phrasebank'
# load dataset
dataset = load_dataset(dataset_name, "sentences_allagree", split = ["train"])[0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [11]:
df = pd.DataFrame(dataset)

In [14]:
df['label'].value_counts()

1    1391
2     570
0     303
Name: label, dtype: int64

In [22]:
min_records = 100
# Create a new dataset with an equal number of records for each label
data_df = pd.concat([df[df['label'] == label].sample(min_records) for label in df['label'].unique()])

# Reset the index of the balanced dataset
data_df.reset_index(drop=True, inplace=True)

In [25]:
conversation_list = []

# Iterate over the DataFrame rows
for index, row in data_df.iterrows():
    # Construct the conversation list dynamically
    conversation = [
        {"from": "human", "value": f"Text: {row['sentence']}"},
        {"from": "phi", "value": "I've read this text."},
        {"from": "human", "value": "Please determine the sentiment of the given text and choose from the options: Positive, Negative, Neutral, or Cannot be determined."}
    ]
    # Append the conversation to the list
    conversation_list.append(conversation)

# Add the conversation list as a new column in the DataFrame
data_df['conversation'] = conversation_list

In [26]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Mit1208/phi-2-classification-sentiment-merged")
model = AutoModelForCausalLM.from_pretrained("Mit1208/phi-2-classification-sentiment-merged", device_map="auto", trust_remote_code=True).eval()

tokenizer_config.json:   0%|          | 0.00/7.61k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [27]:
# {0: 'negative', 1: 'neutral', 2: 'positive'}
def evaluate(y_true, y_pred):
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 2, 'neutral': 1, 'none':1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [28]:
import torch
from transformers import StoppingCriteria
class EosListStoppingCriteria(StoppingCriteria):
    def __init__(self, eos_sequence = tokenizer.encode("<|im_end|>")):
        self.eos_sequence = eos_sequence

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
        return self.eos_sequence in last_ids

In [29]:
def predict(X_test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        inf_conv = X_test.loc[i]['conversation'][:3]
        inference_text = tokenizer.apply_chat_template(inf_conv, tokenize=False) + '<|im_start|>phi:\n'
        inputs = tokenizer(inference_text, return_tensors="pt", return_attention_mask=False, truncation=True).to('cuda')
        outputs = model.generate(inputs["input_ids"], max_new_tokens=1024, pad_token_id= tokenizer.eos_token_id,
            stopping_criteria = [EosListStoppingCriteria()])

        result = tokenizer.batch_decode(outputs)[0]
        answer = result.split("<|im_start|>phi:")[-1].replace("<|im_end|>", "").lower()
        # print(answer)
        if "2" in answer:
            # print("in")
            y_pred.append("positive")
        elif "0" in answer:
            y_pred.append("negative")
        elif "1" in answer:
            # print("n")
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred


In [30]:
data_df.reset_index(inplace=True, drop=True)

In [9]:
data_df.loc[0]['conversation'][:3]

[{'from': 'human',
  'value': "Text: there may not be a critic alive who harbors as much affection for shlock monster movies as i do .\ni delighted in the sneaky - smart entertainment of ron underwood 's big - underground - worm yarn tremors ; i even giggled at last year 's critically - savaged big - underwater - snake yarn anaconda .\nsomething about these films causes me to lower my inhibitions and return to the saturday afternoons of my youth , spent in the company of ghidrah , the creature from the black lagoon and the blob .\ndeep rising , a big - undersea - serpent yarn , does n't quite pass the test .\nsure enough , all the modern monster movie ingredients are in place : a conspicuously multi - ethnic / multi - national collection of bait .\n.. excuse me , characters ; an isolated location , here a derelict cruise ship in the south china sea ; some comic relief ; a few cgi - enhanced gross - outs ; and at least one big explosion .\nthere are too - cheesy - to - be - accidental e

In [31]:
y_pred = predict(data_df, model, tokenizer)

100%|██████████| 300/300 [04:51<00:00,  1.03it/s]


In [32]:
data_df['prd'] = y_pred

In [33]:
data_df['prd'].value_counts()

neutral     100
positive    100
negative    100
Name: prd, dtype: int64

In [34]:
dataset_id2label = {0: 'negative', 2: 'positive', 1:'neutral'}

In [35]:
data_df['label'] = data_df['label'].map(dataset_id2label)

In [ ]:
inf_conv = data_df.loc[1]['conversation']

inference_text = tokenizer.apply_chat_template(inf_conv, tokenize=False) + '<|im_start|>phi:\n'
inputs = tokenizer(inference_text, return_tensors="pt", return_attention_mask=False).to('cuda')
outputs = model.generate(**inputs, max_length=512, pad_token_id= tokenizer.eos_token_id,
            stopping_criteria = [EosListStoppingCriteria()])

text = tokenizer.batch_decode(outputs)[0]

print(text.split("The correct option is")[-1].replace("<|im_end|>", "").replace(".", ""), id2label)

<|im_start|>human
Text: unflinchingly bleak and desperate 
<|im_start|>phi
I've read this text
<|im_start|>human
Please determine the sentiment of the given text and choose from the options: Positive, Negative, Neutral, or Cannot be determined
<|im_start|>phi:
Sentiment of given text is 1 {0: 'negative', 1: 'neutral', 2: 'positive'}


In [36]:
data_df[data_df['label']==data_df['prd']].shape[0]/len(data_df), len(data_df)

(1.0, 300)

In [37]:
class_report = classification_report(data_df['label'], data_df['prd'])
print(class_report)

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00       100
     neutral       1.00      1.00      1.00       100
    positive       1.00      1.00      1.00       100

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



# Reference


1.   https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Perceiver/Fine_tune_Perceiver_for_text_classification.ipynb
2.   https://medium.com/@lukas.hauzenberger/multilabel-classification-using-mistral-7b-on-a-single-gpu-with-quantization-and-lora-8f848b5237f3

